In [21]:
import numpy as np
import plotly.graph_objects as go

from orbslam_processor import ORBSLAMResults, ORBSLAMProcessor
from plotting_utils import create_scattermapbox, create_map_fig
from os import path

TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME))


In [23]:
processor.build_model()

Trial 100 Complete [00h 00m 01s]
val_loss: 1.3788304328918457

Best val_loss So Far: 0.3055846293767293
Total elapsed time: 00h 02m 45s
INFO:tensorflow:Oracle triggered exit
Epoch 1/400
17/17 [==============================] - 0s 1ms/step - loss: 0.3252
Epoch 2/400
17/17 [==============================] - 0s 1ms/step - loss: 0.1662
Epoch 3/400
17/17 [==============================] - 0s 1ms/step - loss: 0.1254
Epoch 4/400
17/17 [==============================] - 0s 1ms/step - loss: 0.1149
Epoch 5/400
17/17 [==============================] - 0s 1ms/step - loss: 0.1072
Epoch 6/400
17/17 [==============================] - 0s 1ms/step - loss: 0.1115
Epoch 7/400
17/17 [==============================] - 0s 1ms/step - loss: 0.0942
Epoch 8/400
17/17 [==============================] - 0s 1ms/step - loss: 0.0864
Epoch 9/400
17/17 [==============================] - 0s 4ms/step - loss: 0.0911
Epoch 10/400
17/17 [==============================] - 0s 2ms/step - loss: 0.0872


In [24]:
processor.fit_trajectories()


46/46 [==============================] - 0s 490us/step


In [30]:
fig_2d = processor.create_2d_plot()
fig_2d.show()